In [1]:
import wrds
import pandas as pd
import numpy as np

In [2]:
# Read tickers from file
with open("data/positive_return_tickers_v1.txt") as f:
    tickers = [line.strip() for line in f if line.strip()]

# Format tickers for SQL IN clause
tickers_str = ",".join([f"'{t}'" for t in tickers])

db = wrds.Connection(wrds_username='changjulian17')

Loading library list...
Done


In [3]:
ASOFDT = '2023-01-03'

# Step 1: Get permnos for tickers
permno_query = f"""
    SELECT DISTINCT ticker, permno
    FROM crsp.dsenames
    WHERE ticker IN ({tickers_str})
"""
permno_df = db.raw_sql(permno_query)
permnos = permno_df['permno'].unique()
permnos_str = ",".join([str(p) for p in permnos])

# Step 2: Get daily volume for those permnos
# """
vol_query = f"""
    SELECT DISTINCT 
        d.date, n.ticker, d.permno, d.vol, d.prc,
        f.mthfloatshrqty AS float_shares,
        ABS(prc) * f.mthfloatshrqty AS market_cap
    FROM crsp.dsf d
    JOIN crsp.dsenames n ON d.permno = n.permno
    LEFT JOIN crsp.stkmthfloatshares f 
    ON d.permno = f.permno 
    AND TO_CHAR(d.date, 'YYYYMM')::int = f.yyyymm
    WHERE d.permno IN ({permnos_str})
    AND d.date = '{ASOFDT}'
    AND n.namedt <= d.date AND n.nameendt >= d.date;
"""
df = db.raw_sql(vol_query)
df = df.sort_values("market_cap", ascending=False)
df.to_csv(f"data/permno_vol_{ASOFDT}.csv", index=False)
df.head()

,date,ticker,permno,vol,prc,float_shares,market_cap
19,2023-01-03,JPM,47896,11054778.0,135.12,2905342.0,392569811.04
4,2023-01-03,BAC,59408,35221501.0,33.51,8009564.0,268400489.64
30,2023-01-03,NEE,24205,5535451.0,83.83,1977540.0,165777178.2
45,2023-01-03,WFC,38703,15944341.0,41.79,3806501.0,159073676.79
38,2023-01-03,SCHW,75186,7443208.0,81.94,1700129.0,139308570.26


In [34]:
tickers_str = "'ZION', 'GS'"
start_date = '2023-01-06'
end_date = '2023-01-06' 
start_time = '09:30:00'
end_time = '16:00:00'

size_query = f"""
    WITH daily_vol AS (
        SELECT
            sym_root,
            date AS trade_date,
            SUM(size) AS daily_volume
        FROM taqm_2023.ctm_2023
        WHERE sym_root IN ({tickers_str})
            AND date BETWEEN '{start_date}' AND '{end_date}'
            AND time_m BETWEEN '{start_time}' AND '{end_time}'
        GROUP BY sym_root, date
    )
    SELECT
        sym_root,
        AVG(daily_volume) AS avg_daily_volume
    FROM daily_vol
    GROUP BY sym_root;
"""
df3 = db.raw_sql(size_query)


In [40]:
tickers_str = "'ZION', 'GS'"
start_date = '2023-01-07'
end_date = '2023-01-15' 
start_time = '09:30:00'
end_time = '16:00:00'

size_query = f"""
    WITH daily_vol AS (
        SELECT
            sym_root,
            date AS trade_date,
            SUM(size) AS daily_volume
        FROM taqm_2023.ctm_2023
        WHERE sym_root IN ({tickers_str})
            AND date BETWEEN '{start_date}' AND '{end_date}'
            AND time_m BETWEEN '{start_time}' AND '{end_time}'
        GROUP BY sym_root, date
    )
    SELECT
        sym_root,
        AVG(daily_volume) AS avg_daily_volume
    FROM daily_vol
    GROUP BY sym_root;
"""
df3 = db.raw_sql(size_query)


In [41]:
df3

,sym_root,avg_daily_volume
0,GS,2193243.8
1,ZION,797272.4
